Let's query some interesting stuff about foods

In [1]:
import pandas as pd # for loading data
import mysql.connector as con


In [2]:
connection = con.connect(user = 'root', password = 'root', database = 'foodb')

I will query the top 20 compounds that are present in most of the foods

In [11]:
compounds = pd.read_sql("SELECT ta.source_id, comp.name, ta.cnt FROM (SELECT source_id, COUNT(*) AS cnt " + 
                    "FROM `contents` WHERE source_type = \"Compound\" GROUP BY source_id ORDER BY cnt DESC LIMIT 20) ta, compounds " + 
                    "AS comp WHERE ta.source_id = comp.id" , connection)

In [12]:
compounds.head()

,source_id,name,cnt
0,14507,Folic acid,28009
1,13831,Retinol,22931
2,21595,Unsaturated fatty acids,18776
3,13267,Cyanocobalamin,12424
4,565,alpha-Tocopherol,11536


Let's rename some of the entries, so we can recognize them more easily

In [36]:
compounds.name = compounds.name.replace(
    ["Retinol", 
     "Cyanocobalamin", 
     "alpha-Tocopherol", 
     "L-Ascorbic acid", 
     "Riboflavine"], 
    ["VitaminA1", "VitaminB12", 
     "VitaminE", "VitaminC", "VitaminB2"]).str.replace(' ', '')

In [37]:
compounds.head()

,source_id,name,cnt
0,14507,Folicacid,28009
1,13831,VitaminA1,22931
2,21595,Unsaturatedfattyacids,18776
3,13267,VitaminB12,12424
4,565,VitaminE,11536
